In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
os.chdir('/Users/hongrudu/Desktop/COVID_Vaccination_rate/data/raw_data')

In [3]:
def create_relative(df, col):
    df_ref = df[['Week_Mon', col]].groupby(['Week_Mon']).mean().reset_index()
    df_ref = df_ref.rename(columns = {col : 'ref'})
    df = df.merge(df_ref, left_on = ['Week_Mon'], right_on = ['Week_Mon'])
    df['Relative_' + col] = df[col]/df['ref']
    df = df.drop(columns = ['ref'])
    return df

In [4]:
ref = pd.read_csv('state_fips_master.csv',
                 dtype = {'fips' : np.str_})
labels = ref[~ref['fips'].isin(['33', '54'])]['state_name']

In [5]:
#### Read raw data
cases = pd.read_pickle('Weekly_cases.pkl')
hospitalized = pd.read_pickle('Weekly_hospitalization.pkl')
df = hospitalized.merge(cases)
#### Smooth data
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Cases']] =\
    df[df['FIPS'] == fips][['Cases']].rolling(window = 3).mean()
df = df.dropna().reset_index().drop(columns = ['index'])
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Cases_lag1']] =\
    df[df['FIPS'] == fips]['Cases'].shift(2)
df = df.dropna().reset_index().drop(columns = ['index'])
df['IHR_lag1'] = df['hospitalized']/df['Cases_lag1']
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['IHR_lag1']] =\
    df[df['FIPS'] == fips][['IHR_lag1']].rolling(window = 3).mean()
pop = pd.read_pickle('State_population.pkl')
df = pop.merge(df)
#### Vaccine Data
vaccine = pd.read_pickle('Weekly_vaccination.pkl')
df = df.merge(vaccine)
for i in vaccine.columns[2:]:
    df[i] = df[i]/df['Population']
for i in vaccine.columns[2:]:
    df[i] = df[i]/7
for fips in ['35', '42']:
    df.loc[df[df['FIPS'] == fips].index, ['People_fully_vaccinated']] =\
    df[df['FIPS'] == fips][['People_fully_vaccinated']].rolling(window = 3).mean()
##### Policy data
policy = pd.read_pickle('Weekly_policy.pkl')
df = df.merge(policy)
###### Testing data
test = pd.read_pickle('Weekly_testing.pkl')
test.Weekly_testing = test.Weekly_testing.mask(test.Weekly_testing.lt(0),np.nan)
test['Weekly_testing'] = test['Weekly_testing'].interpolate(method='linear')
df = df.merge(test)
for i in test.columns[2:]:
    df[i] = df[i]/df['Population']
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Weekly_testing']] =\
    df[df['FIPS'] == fips][['Weekly_testing']].rolling(window = 3).mean()
#### Past infections
past_inf = pd.read_pickle('Weekly_past_infection.pkl')
df = df.merge(past_inf)
for i in past_inf.columns[2:]:
    df[i] = df[i]/df['Population']

In [6]:
data_raw = df.copy()
data_raw = data_raw[data_raw['Week_Mon']>='2021-03-01'].reset_index().drop(columns = ['index'])
data_raw = data_raw[data_raw['Week_Mon']<='2022-02-21'].reset_index().drop(columns = ['index'])

In [7]:
mobility = pd.read_pickle('Weekly_activity_level.pkl')
mobility['Week_Mon'] = pd.to_datetime(mobility['Week_Mon'])
mobility = mobility[mobility['Week_Mon']>='2021-03-01'].reset_index().drop(columns = ['index'])

In [8]:
mobility_cols = ['gym', 'University', 'office_of_physician']
for col in mobility_cols:
    mobility = create_relative(mobility, col)
for col in data_raw.columns[3:]:
    data_raw = create_relative(data_raw, col)

In [9]:
All_states = ['01', '02', '04', '05', '06', '08', '09', '10', '12', '13',
'15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
'26', '27', '28', '29', '30', '31', '32', '34', '35', '36',
'37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48',
'49', '50', '51', '53', '55', '56']

In [10]:
colors = plt.get_cmap('tab20')(np.linspace(0, 1, 48))

In [12]:
fig, axs = plt.subplots(8,2, figsize=(18,15),sharex=True)
i = 0
alpha = 0.8
markersize = 2
for fips in All_states:
    axs[0,0].plot(pd.to_datetime(data_raw[data_raw['FIPS'] == fips]['Week_Mon']), 
             data_raw[data_raw['FIPS'] == fips]['IHR_lag1'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[0,0].set_ylabel('CHR')
    plt.text(0.01, 0.85, "a1", transform=axs[0,0].transAxes, fontstyle = 'italic')
    axs[1,0].plot(pd.to_datetime(data_raw[data_raw['FIPS'] == fips]['Week_Mon']), 
             data_raw[data_raw['FIPS'] == fips]['People_fully_vaccinated'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[1,0].set_ylabel('CPSR')
    plt.text(0.01, 0.85, "b1", transform=axs[1,0].transAxes, fontstyle = 'italic')
    axs[2,0].plot(pd.to_datetime(data_raw[data_raw['FIPS'] == fips]['Week_Mon']), 
             data_raw[data_raw['FIPS'] == fips]['Past_12_week_infections'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[2,0].set_ylabel('PI')
    plt.text(0.01, 0.85, "c1", transform=axs[2,0].transAxes, fontstyle = 'italic')
    axs[3,0].plot(pd.to_datetime(data_raw[data_raw['FIPS'] == fips]['Week_Mon']), 
             data_raw[data_raw['FIPS'] == fips]['GovernmentResponseIndex_Average'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[3,0].set_ylabel('GP')
    plt.text(0.01, 0.85, "d1", transform=axs[3,0].transAxes, fontstyle = 'italic')
    axs[4,0].plot(pd.to_datetime(data_raw[data_raw['FIPS'] == fips]['Week_Mon']), 
             data_raw[data_raw['FIPS'] == fips]['Weekly_testing'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[4,0].set_ylabel('WTR')
    plt.text(0.01, 0.85, "e1", transform=axs[4,0].transAxes, fontstyle = 'italic')
    axs[5,0].plot(pd.to_datetime(mobility[mobility['FIPS'] == fips]['Week_Mon']), 
             mobility[mobility['FIPS'] == fips]['University'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[5,0].set_ylabel('UV')
    plt.text(0.01, 0.85, "f1", transform=axs[5,0].transAxes, fontstyle = 'italic')
    axs[6,0].plot(pd.to_datetime(mobility[mobility['FIPS'] == fips]['Week_Mon']), 
             mobility[mobility['FIPS'] == fips]['gym'], alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[6,0].set_ylabel('GV')
    plt.text(0.01, 0.85, "g1", transform=axs[6,0].transAxes, fontstyle = 'italic')
    axs[7,0].plot(pd.to_datetime(mobility[mobility['FIPS'] == fips]['Week_Mon']), 
             mobility[mobility['FIPS'] == fips]['office_of_physician'], alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[7,0].set_ylabel('PV')
    plt.text(0.01, 0.85, "h1", transform=axs[7,0].transAxes, fontstyle = 'italic')
    
    #####After transformation
    axs[0,1].plot(pd.to_datetime(data_raw[data_raw['FIPS'] == fips]['Week_Mon']), 
             data_raw[data_raw['FIPS'] == fips]['Relative_IHR_lag1'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[0,1].set_ylabel('RCHR')
    plt.text(0.01, 0.85, "a2", transform=axs[0,1].transAxes, fontstyle = 'italic')
    axs[1,1].plot(pd.to_datetime(data_raw[data_raw['FIPS'] == fips]['Week_Mon']), 
             data_raw[data_raw['FIPS'] == fips]['Relative_People_fully_vaccinated'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[1,1].set_ylabel('RCPSR')
    plt.text(0.01, 0.85, "b2", transform=axs[1,1].transAxes, fontstyle = 'italic')
    axs[2,1].plot(pd.to_datetime(data_raw[data_raw['FIPS'] == fips]['Week_Mon']), 
             data_raw[data_raw['FIPS'] == fips]['Relative_Past_12_week_infections'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[2,1].set_ylabel('RPI')
    plt.text(0.01, 0.85, "c2", transform=axs[2,1].transAxes, fontstyle = 'italic')
    axs[3,1].plot(pd.to_datetime(data_raw[data_raw['FIPS'] == fips]['Week_Mon']), 
             data_raw[data_raw['FIPS'] == fips]['Relative_GovernmentResponseIndex_Average'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[3,1].set_ylabel('RGP')
    plt.text(0.01, 0.85, "d2", transform=axs[3,1].transAxes, fontstyle = 'italic')
    axs[4,1].plot(pd.to_datetime(data_raw[data_raw['FIPS'] == fips]['Week_Mon']), 
             data_raw[data_raw['FIPS'] == fips]['Relative_Weekly_testing'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[4,1].set_ylabel('RWTR')
    plt.text(0.01, 0.85, "e2", transform=axs[4,1].transAxes, fontstyle = 'italic')
    axs[5,1].plot(pd.to_datetime(mobility[mobility['FIPS'] == fips]['Week_Mon']), 
             mobility[mobility['FIPS'] == fips]['Relative_University'],alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[5,1].set_ylabel('RUV')
    plt.text(0.01, 0.85, "f2", transform=axs[5,1].transAxes, fontstyle = 'italic')
    axs[6,1].plot(pd.to_datetime(mobility[mobility['FIPS'] == fips]['Week_Mon']), 
             mobility[mobility['FIPS'] == fips]['Relative_gym'], alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[6,1].set_ylabel('RGV')
    plt.text(0.01, 0.85, "g2", transform=axs[6,1].transAxes, fontstyle = 'italic')
    axs[7,1].plot(pd.to_datetime(mobility[mobility['FIPS'] == fips]['Week_Mon']), 
             mobility[mobility['FIPS'] == fips]['Relative_office_of_physician'], alpha=alpha,
             marker = 'o', color = colors[i], label = fips, markersize = markersize)
    axs[7,1].set_ylabel('RPV')
    plt.text(0.01, 0.85, "h2", transform=axs[7,1].transAxes, fontstyle = 'italic')
    i += 1
plt.subplots_adjust(wspace=0.1, hspace=0) 
# plt.figlegend(labels, loc = 'lower center', ncol=5, labelspacing=0.)
fig.legend(labels, loc='lower center', ncol=10)
cols = ['Variables before transformation', 'Variables after transformation']
for ax, col in zip(axs[0], cols):
    ax.set_title(col)

In [13]:
fig.savefig('/Users/hongrudu/Desktop/COVID_Vaccination_rate/figures/Appendix_figure_2.png',
           dpi = 400, bbox_inches='tight')